In [ ]:
from os import listdir
from os.path import isfile, join

import numpy as np
import cv2
import matplotlib.pyplot as plt
import h5py

from segmentation_models import PSPNet
from segmentation_models import FPN
from segmentation_models import Unet
from segmentation_models.segmentation_models.backbones import get_preprocessing

from keras import backend as K
import keras

import spacexyz
%load_ext autoreload
%autoreload 2

K.tensorflow_backend._get_available_gpus()

## Set Parameters

In [ ]:
have_gt = True
# path_to_data = "/scratch2/peilun/relabeled_data/new_val_images"

# if have_gt:
#     path_to_label = "/scratch2/peilun/val_labels"

path_to_data = "/scratch2/peilun/SpaceXYZ/dr/1"
data_files = spacexyz.path2filelist(path_to_data)

path_to_label = "/scratch2/peilun/SpaceXYZ/dr/2"
label_files = spacexyz.path2filelist(path_to_label)

path_to_model = '/scratch2/peilun/SpaceXYZ/Unet_epoch_10.h5'

num_classes = 1+6

input_size = (512, 512, 3)
output_size = (512, 512)


## Load data and model

In [ ]:
####################################################
############# Read in files ########################
####################################################

data_files = spacexyz.path2filelist(path_to_data)
label_files = spacexyz.path2filelist(path_to_label)

data_files.sort()
label_files.sort()

n_samples = len(data_files)

print('Num of images: ', n_samples)

print("reading in ", n_samples, " samples...")
X_ = np.zeros([n_samples, *input_size]).astype(np.uint8)
y_ = np.zeros([n_samples, *input_size[:2]]).astype(np.uint8)
remember_size = np.zeros([n_samples, 2])
for i in range(n_samples):
    print(i, end='. ')
    image = cv2.imread(join(path_to_data, data_files[i]))
    remember_size[i, :] = image.shape[:2]
    X_[i,:,:,:] = cv2.resize(image, input_size[:2])
    if have_gt:
        label = cv2.imread(join(path_to_label, label_files[i]))
        y_[i,:,:] = cv2.resize(label[:,:,0], output_size[:2], interpolation=cv2.INTER_NEAREST)

# y_train = keras.utils.to_categorical(y_train, num_classes=7, dtype='float32')
remember_size = remember_size.astype(np.int64)
assert(remember_size.shape[0] == n_samples)



In [ ]:

####################################################
############# Load model ###########################
####################################################

print("Loading training model...")
model = keras.models.load_model(path_to_model)

## Inference

In [ ]:
####################################################
############# Perform inference ####################
####################################################
print("Performing inference...")
pred = model.predict(X_, batch_size=None, verbose=1, steps=None)

####################################################
############# Resize to original ###################
####################################################


####################################################
############# Save segmentation results ############
####################################################

# pred_labels = []
# for i in range(ind.shape[0]):
# #     ind_original_size =  cv2.resize(ind[i,:,:], tuple(remember_size[i,:2]), interpolation=cv2.INTER_LINEAR)
#     ind_original_size =  cv2.resize(ind[i,:,:], tuple(remember_size[i,:2]))
#     pred_labels.append(ind_original_size)
    
# # todo
# h5f = h5py.File('oxygen_plans_new.h5', 'w')
# h5f.create_dataset('oxygen_plans', data=pred)
# h5f.close()

# # read in results
# h5f = h5py.File('oxygen_plans.h5','r')
# b = h5f['oxygen_plans'][:]

## Visualize

In [ ]:
# todo
h5f = h5py.File('eko_result.h5', 'w')
h5f.create_dataset('eko_result', data=pred)
h5f.close()

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa
import skimage

ind = spacexyz.onehot2ind(pred)
print(ind.shape)

path_to_data = "/scratch2/peilun/SpaceXYZ/dr/1"
data_files = spacexyz.path2filelist(path_to_data)
data_files.sort()

path_to_label = "/scratch2/peilun/SpaceXYZ/dr/2"
label_files = spacexyz.path2filelist(path_to_label)
label_files.sort()

for k in range(n_samples):
    image = cv2.imread(join(path_to_data, data_files[k]))
#     label = ind[k,:,:]
    size = image.shape
    label = cv2.imread(join(path_to_label, label_files[k]))
    label = label[:,:,0]
    print(label.max())
    segmap = label.astype(np.int32)
    label = skimage.transform.resize(label, size[:2], order=0, mode='reflect')
    segmap = ia.SegmentationMapOnImage(segmap, shape=image.shape[:2], nb_classes=num_classes)
#     plt.imshow(segmap.draw_on_image(image))
    s_name = './final_overlay_gt/'+str(k)+'.png'
    cv2.imwrite(s_name,segmap.draw_on_image(image))

In [ ]:
# plt.imshow(pred[0,:,:,1])
# plt.imsave('wall.png', pred[0,:,:,1])



In [ ]:
k=83
label = pred_labels[k]
segmap = label.astype(np.int32)
segmap = ia.SegmentationMapOnImage(segmap, shape=(512, 512), nb_classes=1+6)
# plt.imshow(segmap.draw_on_image(X_[k,:,:,:]))
plt.imshow(segmap.draw_on_image(t_image))
cv2.imwrite('messigray.png',segmap.draw_on_image(X_[k,:,:,:]))